In [1]:
import numpy as np, pandas as pd, plotly.express as px, plotly.graph_objects as go
from pathlib import Path

from SJ_Haar_CNV import base_search
from SJ_Haar_CNV import decomposition
from tests import test_data

In [13]:
X = np.concatenate([np.repeat (5,130), np.repeat (7,190), np.repeat(5,100)]) + np.random.normal(0,1,420)
X

array([5.3667742 , 4.85782335, 4.8167225 , 4.8588647 , 4.81747854,
       4.67083285, 4.20384477, 3.87452254, 5.47303802, 4.93237007,
       3.26845437, 4.37722478, 4.30526237, 4.74830481, 4.73184891,
       6.45308417, 5.20865349, 5.17151852, 3.77436106, 4.79217456,
       5.59199342, 4.56652143, 3.94487235, 5.11859833, 6.27668762,
       5.42043712, 4.98164828, 5.12110727, 4.66050604, 4.10090686,
       4.72946241, 5.11044578, 5.44270859, 4.93969109, 3.82589718,
       4.76115526, 5.36725541, 4.29447841, 4.50317035, 4.49857485,
       5.26133603, 4.56226607, 4.94137279, 5.18749278, 4.6283875 ,
       5.40621194, 6.95259618, 6.40586724, 5.366835  , 7.95347727,
       6.72358992, 6.26302018, 5.79418392, 4.90537798, 5.9872164 ,
       5.0072751 , 4.33573204, 3.86790592, 5.74755897, 4.47741165,
       4.70966609, 5.32200286, 5.27481885, 4.49607858, 3.40258813,
       5.79765441, 4.87121407, 3.65951454, 3.87639778, 5.27435407,
       5.45799854, 3.87516241, 3.95166828, 4.47477625, 7.35939

In [14]:
basis = base_search.generate_haar_basis(X, p0=.95, length = 20, debug = False)
basis

[[0, 0, 0, (5.911882037722706, 420), (0, 0)],
 [0, 420, 0, (0.07251407362812771, 130), (-0.032756357397533555, 290)],
 [1, 130, 0, (0.25257668863597027, 13), (-0.030222851631654585, 117)],
 [1, 289, 131, (0.042601432284230495, 189), (-0.08094272134003795, 100)],
 [2, 116, 14, (0.30241195653113895, 9), (-0.02826279967580735, 107)],
 [2, 189, 131, (0.19548683580954593, 22), (-0.02692333666838059, 167)],
 [2, 99, 321, (0.21343747458109494, 17), (-0.04685212856658182, 82)],
 [3, 106, 24, (0.22243591216177602, 16), (-0.042015672297224374, 90)],
 [3, 22, 131, (0.5383819020581655, 2), (-0.08075728530872485, 20)],
 [3, 166, 154, (0.2914121942489626, 10), (-0.020548295748324272, 156)],
 [3, 81, 339, (0.3358643744858637, 7), (-0.03630966210657986, 74)],
 [4, 89, 41, (0.39440531887330776, 5), (-0.028171808490950533, 84)],
 [4, 155, 165, (0.323575114464717, 8), (-0.01981072129375818, 147)],
 [4, 73, 347, (0.43184081151100867, 4), (-0.03129281242833395, 69)],
 [5, 83, 47, (0.12297261940068255, 36),

In [15]:
res = decomposition.decompose (X, basis)

In [19]:
px.scatter (X)
px.line (decomposition.generate_function_from_wavelets (res.x, basis))

In [5]:
a = np.ones(420)

a[0:420] = a[0:420] + np.ones(420)

In [6]:
a

array([2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2.

In [7]:
X.shape

(420,)